Hello Databricks!


In [0]:
print("Starting first databricks here. Conncted to GitHub Personal Repo...")

In [0]:
def print_hello(name):
    print(f"Hello {name}..")
print_hello(name="Onkar")

In [0]:
some_data = [{"name": "Onkar", "age": 30, "salary": 100000, "city" : "Helsingborg","role":"Data Engineer","department":"Range Ops"}]
schema = "name string, age int, salary int, city string, role string, department string"
df = spark.createDataFrame(some_data, schema=schema)
display(df)
df.printSchema()